In [ ]:
#|default_exp optimizer.adan

# Adan: ADAptive Nesterov Momentum Optimizer
> Adds the Adan optimizer to fastai from [Adan: Adaptive Nesterov momentum Algorithm for Faster Optimizing Deep Models](https://arxiv.org/abs/2208.06677).

In [ ]:
#|export
from __future__ import annotations
from typing import Optional, Dict

from fastai.optimizer import Optimizer, step_stat

from fastxtend.optimizer.torchscript import JitOptimizer
from fastxtend.imports import *

In [ ]:
#|hide
from fastxtend.test_utils import *

In [ ]:
#|exporti
def debias(beta:float, step:int):
    "Simple debias calculation"
    return 1-beta**step

## Adan Fastai Callbacks -

In [ ]:
#|exporti
def adan_setup(p, step=0, grad_avg=None, diff_avg=None, nesterov_est=None, prior_grad=None, paper_init=False, **kwargs):
    "Handles Adan setup and keeps track of steps"
    if step == 0: 
        grad_avg     = torch.zeros_like(p, memory_format=torch.preserve_format)
        diff_avg     = torch.zeros_like(p, memory_format=torch.preserve_format)
        nesterov_est = torch.zeros_like(p, memory_format=torch.preserve_format)
        if paper_init:
            prior_grad = p.grad.clone()
        else:
            prior_grad = torch.zeros_like(p, memory_format=torch.preserve_format)
        step += 1
        return {'grad_avg':grad_avg, 'diff_avg':diff_avg, 'nesterov_est':nesterov_est, 'prior_grad':prior_grad, 'step':step}
    else:
        step += 1
        return {'step':step}

In [ ]:
#|exporti
def adan_avgs(p, beta1, beta2, beta3, grad_avg, diff_avg, nesterov_est, prior_grad, **kwargs):
    "Updates Adan moving averages"
    # update m_k
    grad_avg.mul_(beta1).add_(p.grad.data, alpha=1-beta1)

    # update v_k
    diff_avg.mul_(beta2).add_(p.grad.data-prior_grad, alpha=1-beta2)
    
    # update n_k
    nesterov_est.mul_(beta3).add_(torch.square(torch.add(p.grad.data, torch.sub(p.grad.data, prior_grad), alpha=beta2)), alpha=1-beta3)

    # set current grad as next step's prior_grad
    prior_grad = p.grad.data.clone()
    return {'grad_avg':grad_avg, 'diff_avg':diff_avg, 'nesterov_est':nesterov_est, 'prior_grad':prior_grad}

adan_avgs.defaults = dict(beta1=0.98, beta2=0.92, beta3=0.99)

In [ ]:
#|exporti
def adan_step(p, lr, eps, wd, beta1, beta2, beta3, step, grad_avg, diff_avg, nesterov_est, do_wd=True, **kwargs):
    "Performs the Adan step with `lr` on `p`"
    # calculate debias terms
    db1, db2, db3 = debias(beta1, step), debias(beta2, step), debias(beta3, step)

    # calculate applied λ 
    wd = (1+lr*wd) if wd!=0 and do_wd else 1 

    # calculate η_k
    lr = lr/torch.sqrt(nesterov_est/db3+eps) 

    # perform Adan step and apply to parameter `p`
    p.data.sub_(torch.add(grad_avg/db1, diff_avg/db2, alpha=beta2).mul_(lr)).div_(wd)
    return p

## Adan Jit -

In [ ]:
#|exporti
@torch.jit.script
def adan_jit_step(p:Tensor, grad:Tensor, lr:float, wd:float, beta1:float, beta2:float, beta3:float, eps:float, paper_init:bool,
                  grad_avg:Optional[Tensor]=None, diff_avg:Optional[Tensor]=None, nesterov_est:Optional[Tensor]=None, 
                  prior_grad:Optional[Tensor]=None, do_wd:bool=True, step:int=0, force_train:Optional[bool]=None, mom:Optional[float]=None):
    dp = p
    step += 1

    if grad_avg is None: 
        grad_avg = torch.zeros_like(dp, memory_format=torch.preserve_format)
    if diff_avg is None: 
        diff_avg = torch.zeros_like(dp, memory_format=torch.preserve_format)
    if nesterov_est is None: 
        nesterov_est = torch.zeros_like(dp, memory_format=torch.preserve_format)
    if prior_grad is None:
        if paper_init:
            prior_grad = grad.clone()
        else:
            prior_grad = torch.zeros_like(dp, memory_format=torch.preserve_format)

    # update m_k
    grad_avg = torch.mul(grad_avg, beta1)
    grad_avg = torch.add(grad_avg, grad, alpha=1-beta1)

    # update v_k
    diff_avg = torch.mul(diff_avg, beta2)
    diff_avg = torch.add(diff_avg, grad-prior_grad, alpha=1-beta2)
    
    # update n_k
    nesterov_est = torch.mul(nesterov_est, beta3)
    nesterov_est = torch.add(nesterov_est, torch.square(torch.add(grad, torch.sub(grad, prior_grad), alpha=beta2)), alpha=1-beta3)

    # calculate debias terms
    db1 = debias(beta1, step)
    db2 = debias(beta2, step)
    db3 = debias(beta3, step)

    # calculate applied λ
    if wd!=0 and do_wd:
        wd = (1+lr*wd) 
    else:
        wd = 1. 

    # calculate η_k
    lr = lr/torch.sqrt(nesterov_est/db3+eps) 

    # perform Adan step
    dp = torch.sub(dp, torch.mul(torch.add(grad_avg/db1, diff_avg/db2, alpha=beta2), lr))
    dp = torch.div(dp, wd)

    # set current grad as next step's prior_grad
    prior_grad = grad.clone()

    # apply results to parameter p
    p.set_(dp)

    return torch.jit.annotate(Dict[str, Union[Tensor, int]], {'grad_avg':grad_avg, 'diff_avg':diff_avg, 'nesterov_est':nesterov_est, 'prior_grad':prior_grad, 'step':step})

## Adan -

In [ ]:
#|export
def Adan(params, lr, beta1=0.98, beta2=0.92, beta3=0.99, eps=1e-8, wd=0.02, paper_init=False, jit=False):
    "A `Optimizer` for Adan with `lr`, `beta`s, `eps` and `params`"
    if jit:
        cb = partial(adan_jit_step, paper_init=paper_init)
        return JitOptimizer(params, cb, lr=lr, beta1=beta1, beta2=beta2, beta3=beta3, eps=eps, wd=wd)
    else:
        cbs = [partial(adan_setup, paper_init=paper_init), adan_avgs, adan_step]
        return Optimizer(params, cbs, lr=lr, beta1=beta1, beta2=beta2, beta3=beta3, eps=eps, wd=wd)

In [ ]:
#|hide
def tst_param(val, grad=None):
    "Create a tensor with `val` and a gradient of `grad` for testing"
    res = tensor([val]).float()
    res.grad = tensor([val/10 if grad is None else grad]).float()
    return res

def tst_params():
    r = L.range(4)
    return r.map(tst_param)

params_org = tst_params()
opt_org = Adan(params_org, lr=0.1, jit=False)
opt_org.step()

params_jit = tst_params()
opt_jit = Adan(params_jit, lr=0.1, jit=True)
opt_jit.step()

test_close([p.item() for p in params_org], [p.item() for p in params_jit])

opt_org.step()
opt_jit.step()
test_close([p.item() for p in params_org], [p.item() for p in params_jit])

In [ ]:
#|export
def AdanLargeBatchLR(bs):
    "Square root rule for scaling `Adan` learning rate for large-batch training"
    return math.sqrt(bs/256)*6.25e-3